# 0. Problem
  
1. Har det hamnat siffror i kolumnen enkätexter? Kanske för variabler som är tagna från register. Ersätt dessa med NaN isåfall.

***  

På [den här sidan](https://github.com/atlanhq/camelot) finns bakgrundsinformation.

## 1. Bibliotek  
Jag behöver pandas, re (för regex) och camelot. Camelot är modulen som hämtar "tabeller" fårn pdf-filer.

In [40]:
import camelot
import pandas as pd
import re
import numpy as np

## 2. Camelot  
Läs in PDF-filen. Just den här filen har tre sidor. Camelot ska lägga "tabellerna" från pdf:en i en lista. Av någon anledning hämtar den bara tabellen från första sidan.

In [2]:
tables = camelot.read_pdf('LNU32.pdf')
tables

<TableList n=1>

Gör om "tabellen" till en DataFrame (`df`). Det verkar som att varje ruta i LNU:s kodbok blir en cell i `df` (som bara har en kolumn). Värdet i cellen är en sträng. Camelot tycks ha plockat varje textbit i rutan och lagt ihop till en lång sträng. Delarna i strängen skiljs åt av "\n".  

In [3]:
tables.n

1

In [4]:
df = tables[0].df
df

,0
0,"Z8 \nTYP AV INTERVJU \nX10, Y7, U7 \nTyp av in..."
1,"Z9 \nINDIREKT INTERVJU \njfr X11, Y8, \nU8, V7..."
2,Z10 \nINTERVJUSPRÅK \njfr X12 \nAnger huruvida...
3,"Z11 \nKÖN \nX8, Y10, U10, \nV718, W2 \nIntervj..."
4,"Z12 \nFÖDELSEÅR \nX9, Y11, U11, \nV815, W533 \..."


Såhär ser den första cellen ut.

In [5]:
df.loc[0][0]

'Z8 \nTYP AV INTERVJU \nX10, Y7, U7 \nTyp av intervju. \n1 \nBesöksintervju \n3278 \n74.3 \n2 \nTelefonintervju \n1137 \n25.7 \n \nTotalt \n4415 \n100'

Nu skapar jag en lista `L1` som innehåller strängen från varje cell i `df`. Varje sådan sträng har gjorts om till en lista (`L1[i]`) med kortare strängar som tagits fram genom att dela den ursprungliga strängen vid varje "\n". Varje `L1[i]` motsvarar alltså en variabel i LNU:s kodbok.

In [6]:
L1 = [df.loc[i][0].split('\n') for i in df.index]

## 3. Kolumner  
Här förklaras koden som skapar de tre kolumnerna **Variabelnamn**, **Beskrivning** och **Enkättext**. 

### 3.1 Variabelnamn   
Den första strängen i varje ruta i kodboken (`L1[i]`) är oftast variabelnamnet.

In [7]:
L1[0][0]

'Z8 '

Vid sidbrytning kommer dock den första strängen inte se ut sådär. Därför måste jag använda ett `regex`. Alla variabelnamn i LNU 2010 börjar på "Z" och följs av åtminstone en siffra, eller så börjar de med "PZ" och följs av åtminstone en siffra.

In [8]:
regex1 = re.compile(r'^Z\d|^PZ\d')

Listan `namn` ska innehålla alla variabelnamn. Jag går igenom den första strängen i varje cell och ser ifall den är ett "äkta" variabelnamn med hjälp av `regex1`. Är den det läggs strängen i `namn`, i annat fall lägger jag `'sidbrytning'` i listan. På så vis kan jag i ett senare skede ta bort alla rader i tabellen vars variabelnamn är `'sidbrytning'`.

In [9]:
namn = []

for i in range(len(L1)):
    for j in range(len(L1[i])):
        a = len(namn)
        if bool(regex1.findall(L1[i][j])) == True:
            namn.append(L1[i][j])
            break
    if len(namn) == a:
        namn.append('sidbrytning')

### 3.2 Beskrivning  
Den andra strängen är alltid (utom vid sidbrytningar så klart) en beskrivning.

In [10]:
L1[1][1]

'INDIREKT INTERVJU '

På samma sätt går det därför att samla alla beskrivningar i en lista. Jag använder en regex som fångar strängar som innehåller åtminstone tre stora bokstäver i följd. Samtliga beskrivningar i LNU:s kodbok är nämligen gjorda med stora bokstäver.

In [ ]:
regex2 = re.compile(r'[A-Z]{3}')

Jag går igenom varje sträng i `L1` och plockar ut den första strängen som fångas av `regex2`. Hittar den ingen sådan sträng skriver den 'sidbrytning' istället.

In [13]:
beskrivning = []

for i in range(len(L1)):
    for j in range(len(L1[i])):
        a = len(beskrivning)
        if bool(regex2.findall(L1[i][j])) == True:
            beskrivning.append(L1[i][j])
            break
    if len(beskrivning) == a:
        beskrivning.append('sidbrytning')


    

In [14]:
beskrivning

['TYP AV INTERVJU ',
 'INDIREKT INTERVJU ',
 'INTERVJUSPRÅK ',
 'sidbrytning',
 'FÖDELSEÅR ']

### 3.3 Enkättext  
Det är mer komplicerat att hitta enkättexten. Den kommer alltid på tredje, fjärde eller femte plats beroende på hur många "liknande variabler" som listats i högra hörnet i kodboken. För att hantera detta skapar jag en regex som svarar på strängar som beskriver "liknande variabler". Sådana strängar undviks och de övriga läggs i en lista. Enkättexten kan även sträcka sig över flera rader i kodboken. I `L1` blir den därför olika strängar som måste sättas ihop igen. Efter "sista" enkättext-strängen börjar alltid kodlistorna. Eftersom dessa alltid börjar med siffror (oftast 1 men ibland även årtal som 1942) kan jag använda ytterligare et regex.

In [17]:
regex3 = re.compile(r'^NY|^jfr|^[XYZUVW]\d|^\s')

regex4 = re.compile(r'^\d')

In [18]:
enkättext = []

for i in range(len(L1)):
    textdelar = []
    for j in range(3, len(L1[i])):
        if bool(regex4.findall(L1[i][j])) == True:
            break
        if bool(regex3.findall(L1[i][j])) == False:
            textdelar.append(L1[i][j])
    enkättext.append(''.join(textdelar))

In [19]:
enkättext

['Typ av intervju. ',
 'Anger huruvida intervju gjordes med intervjupersonen eller annan än intervjupersonen ',
 'Anger huruvida intervjun genomfördes på svenska eller med hjälp av tolk. ',
 'Intervjupersonens kön. ',
 'Intervjupersonens födelseår. ']

Tyvärr verkar det som att vissa variabler saknar enkättext (t.ex. Z3). Dessa är förmodligen väldigt få, de är sanolikt bara vissa bakgrundsvariabler som saknar enkättext. I dessa fall läggs det en siffran från variabelns kodlista i `enkättext`. Detta går enkelt att åtgärna i efterhand genom att ersätta alla siffror med säg `NaN`.

## 4. Skapa tabellen  
Här kommer all kod ovan samlad i en cell. Anledningen är att vissa sidor i kodboken saknar tabell helt. Detta testas i loopen som går igenom alla sidor i kodboken. Loopen skapar en lista som heter "delar". Den innehåller en dataframe för varje sida i kodboken.

In [26]:
regex1 = re.compile(r'^Z\d|^PZ\d')

regex2 = re.compile(r'[A-Z]{3}')

regex3 = re.compile(r'^NY|^jfr|^[XYZUVW]\d')

regex4 = re.compile(r'^\d')

delar = []

for i in range(32, 357):
    # Läs in sidan i från LNU:s kodbok
    tables = camelot.read_pdf('LNU{}.pdf'.format(i))
    
    print(i)
    # Gör om "tabellen" på sidan till en DataFrame. Vissa sidor, t.ex. 36, har ingen tabell alls. Kolla därför att tabell finns.
    if tables.n != 0:
        df = tables[0].df

        # Dela upp strängen i varje cell vid "\n".
        L1 = [df.loc[i][0].split('\n') for i in df.index]
    
        # Skapa lista med namn (se exempel längre upp för förklaring av regex)
        namn = []

        for i in range(len(L1)):
            for j in range(len(L1[i])):
                a = len(namn)
                if bool(regex1.findall(L1[i][j])) == True:
                    namn.append(L1[i][j])
                    break
            if len(namn) == a:
                namn.append('sidbrytning')
        
        
        # Skapa listan med beskrivningar.         
        beskrivning = []

        for i in range(len(L1)):
            for j in range(len(L1[i])):
                a = len(beskrivning)
                if bool(regex2.findall(L1[i][j])) == True:
                    beskrivning.append(L1[i][j])
                    break
            if len(beskrivning) == a:
                beskrivning.append('sidbrytning')
        
        # Skapa listan med enkättexter.
        enkättext = []

        for i in range(len(L1)):
            textdelar = []
            for j in range(3, len(L1[i])):
                if bool(regex4.findall(L1[i][j])) == True:
                    break
                if bool(regex3.findall(L1[i][j])) == False:
                    textdelar.append(L1[i][j])
            enkättext.append(''.join(textdelar))
        
        # Skapa en dataframe av listorna namn, beskrivning och enkättext. 
        df = pd.DataFrame.from_dict({'Variabelnamn': namn, 'Beskrivning': beskrivning, 'Enkättext': enkättext})
    
        # Nu har hela sidan i blivit en dataframe och läggs i en lista som heter "delar".
        delar.append(df)

32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298


Såhär kan en "del" i `delar` se ut. Dessa måste nu slås samman till en stor dataframe.

In [250]:
delar[189]

,Variabelnamn,Beskrivning,Enkättext
0,sidbrytning,sidbrytning,
1,Z597,ARBETSLÖS UNDER 2009,"Sökte eller väntade du på arbete, var arbetslö..."
2,Z598,ARBETSLÖSHETSVECKOR 2009,Antal veckor under 2009: arbetssökande/arbetsl...
3,Z599,FÖRRA VECKAN: HELTID,"Var du anställd på heltid (inklusive semester,..."


Lägg alla DataFrames till en stor. Varje liten representerara en sida i kodboken.

In [69]:
LNU = pd.concat(delar)

Ta bort de rader där variabelnamnet är 'sidbrytning'.

In [70]:
LNU = LNU[LNU.Variabelnamn != 'sidbrytning']

Trots detta finns det två variabler med beskrivningen 'sidbrytning'. Förklaringen är att deras beskrivning är "KÖN". Mitt `regex4` fångar inte upp detta eftersom "Ö" är en svensk bokstsav. 

In [71]:
LNU[LNU['Beskrivning'] == 'sidbrytning']

,Variabelnamn,Beskrivning,Enkättext
3,Z11,sidbrytning,Intervjupersonens kön.
2,PZ2,sidbrytning,Är du man eller kvinna?


Variabeln Z11 ska ha beskrivningen "KÖN" och PZ2 ska ha "P-KÖN".

In [77]:
LNU.loc[LNU.Variabelnamn == 'Z11 ', ['Beskrivning']] = 'KÖN'

LNU.loc[LNU.Variabelnamn == 'PZ2 ', ['Beskrivning']] = 'P-KÖN'

Efter sammanslagningen är indexet i oordning. Skapa en ny kolumn med heltal och gör den till index.

In [78]:
LNU = LNU.reset_index()

LNU.set_index('index', inplace = True)

Lägg till en kolumn som säger vilken enkät variabeln kommer ifrån och fyll kolumnen med 'LNU'.

In [79]:
LNU['Enkät'] = 'LNU'

Kolumnen med itemnr innehåller ingen information för LNU-variabler eftersom dessa skapas genom besöksintervju. Lägg därför `NaN` i hela kolumnen. Ändra slutligen ordning en på kolumnerna.

In [80]:
LNU['Itemnr'] = np.nan

LNU = LNU[['Variabelnamn', 'Enkät', 'Beskrivning', 'Itemnr', 'Enkättext']]

Strängarna i `LNU` innehåller whitespace lite överallt i början och slutet. Jag tar bort detta eftersom det kan vara förvirrande när man t.ex. vill hämta en viss variabel.

In [86]:
LNU['Variabelnamn'] = LNU['Variabelnamn'].str.strip()

LNU['Beskrivning'] = LNU['Beskrivning'].str.strip()

LNU['Enkättext'] = LNU['Enkättext'].str.strip()

Allt verkar se bra ut.

In [81]:
LNU.head()

,Variabelnamn,Enkät,Beskrivning,Itemnr,Enkättext
index,,,,,
0,Z8,LNU,TYP AV INTERVJU,NaN,Typ av intervju.
1,Z9,LNU,INDIREKT INTERVJU,NaN,Anger huruvida intervju gjordes med intervjupe...
2,Z10,LNU,INTERVJUSPRÅK,NaN,Anger huruvida intervjun genomfördes på svensk...
3,Z11,LNU,KÖN,NaN,Intervjupersonens kön.
4,Z12,LNU,FÖDELSEÅR,NaN,Intervjupersonens födelseår.


## 5. Skapa .csv

In [32]:
LNU.to_excel('test.xlsx')